In [9]:
from requests import get
from pandas import DataFrame, to_datetime
from plotly.subplots import make_subplots

In [10]:
url = "https://s.cafef.vn/Ajax/PageNew/DataHistory/PriceHistory.ashx?Symbol=VNM&PageSize=1250"
parsed_data_json = get(url).json()['Data']
parsed_data_df = DataFrame(parsed_data_json['Data'])

In [11]:
data = parsed_data_df.assign(
    date=to_datetime(parsed_data_df['Ngay'], dayfirst=True),
    open=parsed_data_df['GiaMoCua'],
    close=parsed_data_df['GiaDongCua'],
    high=parsed_data_df['GiaCaoNhat'],
    low=parsed_data_df['GiaThapNhat'],
    volume=parsed_data_df['KhoiLuongKhopLenh'] +
           parsed_data_df['KLThoaThuan'],
)[['date', 'open', 'close', 'high', 'low', 'volume']]

print(data)

           date   open  close   high    low   volume
0    2024-10-15   67.2   66.8   67.4   66.7  3073300
1    2024-10-14   68.1   67.2   68.1   67.1  3220500
2    2024-10-11   68.1   67.7   68.2   67.5  2690038
3    2024-10-10   68.3   67.7   68.8   67.7  2822615
4    2024-10-09   68.3   67.8   68.6   67.8  3527960
...         ...    ...    ...    ...    ...      ...
1246 2019-10-21  132.5  133.6  135.5  131.9  2006630
1247 2019-10-18  131.5  132.5  133.0  131.5  1173840
1248 2019-10-17  130.4  131.0  132.4  130.2  1430520
1249 2019-10-16  130.4  130.4  130.9  128.3  1481580
1250 2019-10-15  128.5  128.3  129.5  127.7  1393260

[1251 rows x 6 columns]


In [12]:
data.to_csv("vnm_stock_data.csv", index=False)

In [30]:
figure = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=(0.7, 0.3)
)

figure.add_candlestick(
    row=1, col=1,
    name="Price",
    x=data['date'],
    close=data['close'],
    open=data['open'],
    high=data['high'],
    low=data['low']
)

figure.add_bar(
    row=2, col=1,
    name="Volume",
    x=data['date'],
    y=data['volume'],
    marker_color='cyan',
    marker_line_width=0.15
)

figure.update_layout(
    title="VNM Stock Prices and Volume from 2019 to 2024",
    xaxis_title="Date",
    yaxis_title="Price",
    yaxis2_title="Volume",
    xaxis_rangeslider_visible=False,
    xaxis2_rangeslider_visible=True,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='black')
)

figure.show()

In [14]:
html_export = figure.to_html()
with open("vnm_stock_graph.html", 'w', encoding='utf-8') as export_file:
    export_file.write(html_export)